In [ ]:
pip install torch transformers datasets peft evaluate py7zr nltk absl-py rouge_score bert_score


Importing all the libraries needed

In [1]:
import torch as t
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from pathlib import Path
import transformers, datasets, pickle, multiprocessing, peft, evaluate, py7zr, functools 


Create global devices so the gpu can be used

In [2]:
output_device = t.device('cpu')
model_run_device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')

The maximum input length for t5 is 512 tokens. CNN articles are often larger than that. Luckily, there is still more than double the number of samsum dialogues which are below that threshold, so we can just filter those out:

In [5]:
with open(f'./preprocessing/mixed_tokens.pickle', "rb") as file:
    mixed_tokens = pickle.load(file)

Creating the configuration for LoRA and the trainer

In [6]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16, 
    target_modules= ['q', 'v'],  
    lora_dropout=0.1,  
)
base_model = T5ForConditionalGeneration.from_pretrained("t5-base")
lora_model = get_peft_model(base_model, lora_config).to(model_run_device)
training_args = TrainingArguments(
    output_dir="./checkpoints/t5_summarization_lora_mixed",
    eval_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    log_level="info",

    save_total_limit=1,
    overwrite_output_dir=True,
    disable_tqdm=False,
    use_cpu=False,
    fp16=True
)
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=mixed_tokens['train'],
    eval_dataset=mixed_tokens['validation'],
    tokenizer=T5Tokenizer.from_pretrained("t5-base")
)
trainer.model.to(model_run_device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/tmp/ipykernel_1049595/558205819.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


PeftModel(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_e

Train the model!!

In [7]:
trainer.train()
with open(f'./models/mixed_trainer.pickle', 'wb') as file:
    pickle.dump(trainer, file)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.246300,No log
2,0.523100,No log
3,0.509900,No log


FileNotFoundError: [Errno 2] No such file or directory: './models/lora-fine-tuned-mixed_trainer.pickle'

In [18]:
with open(f'./models/lora-fine-tuned-mixed.pickle', 'rb') as file:
    trainer.model = pickle.load(file).to(model_run_device)
trainer.model.to(model_run_device) 

NameError: name 'trainer' is not defined